In [9]:
import numpy as np
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, AveragePooling2D
from keras import Sequential, optimizers
from keras import backend as K
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import random
import copy

In [10]:
data = np.load("chestmnist.npz")
train_data = data["train_images"]
val_data = data["val_images"]
test_data = data["test_images"]
train_labels = data["train_labels"]
val_labels = data["val_labels"]
test_labels = data["test_labels"]

In [11]:
print(np.shape(data))
print(np.shape(data["train_images"]))

(6,)
(78468, 28, 28)


In [12]:
random.seed(0)
np.random.seed(0)

In [13]:
def get_counts(labels, class_num):
    counts = np.zeros(class_num)
    for label in labels:
        for i in np.where(label != 0)[0]:
            counts[i]+=1
    # counts = [i/len(labels) for i in counts]

    return counts

def get_equal_indicies(labels, class_equal):
    indicies = [i for i in range(np.shape(labels)[0]) if labels[i][class_equal] == 1]
    return indicies

def remove_class(data, labels, class_to_remove):
    chosen_indicies = set(get_equal_indicies(labels, class_to_remove))
    remaining_indicies = [i for i in range(np.shape(data)[0]) if i not in chosen_indicies]
    data, labels = data[remaining_indicies], labels[remaining_indicies]
    labels = np.delete(labels, (class_to_remove), axis=1)
    return data, labels

def add_no_class(labels):
    to_append = np.array([np.array([1]) if (label == np.zeros(np.shape(label)[0])).all() else np.array([0]) for label in labels])
    labels = np.append(labels, to_append, axis=1)
    return labels

class_num = np.shape(train_labels)[1]
# print(np.shape(train_labels))
train_counts = get_counts(train_labels, class_num+1)
train_data, train_labels = remove_class(train_data, train_labels, class_num-1)
train_counts = get_counts(train_labels, class_num)
class_num-=1

In [14]:
def split_data(dataset, labels, class_num):
    split_labels = []
    split_data = []

    for i in range(class_num):
        split_data.append(copy.deepcopy(dataset))
        to_add = [1 if label[i] == 1 else 0 for label in labels]
        split_labels.append(to_add)
        
    return split_data, split_labels 

split_train_images, split_labels = split_data(train_data, train_labels, class_num)

In [15]:
def shuffle_together(data, labels):
    p = np.random.permutation(len(data))
    return data[p], labels[p]

# def subsample_class(data, labels, majority_class):
#     all_indices = np.array(get_equal_indicies(labels, majority_class))

#     excluded_indices = [all_indices[(random.randint(0,len(all_indices)-1))] for i in range(len(all_indices))]
#     excluded_indicies = set(excluded_indices)
#     chosen_indicies = np.array([i for i in range(np.shape(data)[0]) if i not in excluded_indicies])

#     return data[chosen_indicies], labels[chosen_indicies]

# def oversample_class(data, labels, minority_class, copy_num, class_num):
#     all_indices = [i for i in get_equal_indicies(labels, minority_class) if list(labels[i]).count(1) == 1]
#     data_to_add = np.zeros((copy_num, 28, 28))
#     labels_to_add = np.zeros((copy_num, class_num), dtype=int)

#     for i in range(copy_num):
#         rnd_index = random.randint(0,len(all_indices)-1)
#         data_to_add[i] = data[all_indices[rnd_index]].copy()
#         labels_to_add[i] = labels[all_indices[rnd_index]].copy()

#     # labels_to_add = np.reshape(labels_to_add, (np.shape(labels_to_add)[0]))
#     data = np.append(data, data_to_add, axis=0)
#     labels = np.append(labels, labels_to_add, axis=0)

#     return data, labels

# def oversample_binary(data, labels, copy_num):
#     chosen_indicies = [i for i in range(np.shape(data)[0]) if labels[i] == 1]
#     data_to_add = np.zeros((copy_num, 28, 28))

#     for i in range(copy_num):
#         rnd_index = random.randint(0,len(chosen_indicies)-1)
#         data_to_add[i] = data[chosen_indicies[rnd_index]].copy()
    
#     data = np.append(data, data_to_add, axis=0)
#     labels = np.append(labels, np.ones(copy_num))

#     data, labels = shuffle_together(data, labels)
#     return data, labels

def balance_dataset(split_dataset, split_labels):
    # initialise samplers
    ros = RandomOverSampler(random_state=0)
    rus = RandomUnderSampler(random_state=0)
    
    # iterate through each class to balance each dataset
    for class_index in range(len(split_dataset)):
        split_labels[class_index] = np.array(split_labels[class_index])

        # reshape as over samplers require 2 dimensional arrays
        split_dataset[class_index] = split_dataset[class_index].reshape((78324,784))

        # oversample dataset
        split_dataset[class_index], split_labels[class_index] = ros.fit_resample(split_dataset[class_index], split_labels[class_index])
        print(np.shape(split_dataset[class_index]))
        length = np.shape(split_dataset[class_index])[0]

        # reshape back into 3 dimensional array
        split_dataset[class_index] = split_dataset[class_index].reshape((length,28,28))

        split_dataset[class_index], split_labels[class_index] = shuffle_together(split_dataset[class_index], split_labels[class_index])

    return split_dataset, split_labels


split_train_images, split_labels = balance_dataset(split_train_images, split_labels)

(140702, 784)
(152752, 784)
(138144, 784)
(128864, 784)
(148704, 784)
(147910, 784)
(154696, 784)
(149252, 784)
(150130, 784)
(153274, 784)
(153050, 784)
(154340, 784)
(152102, 784)


In [16]:

# def loss(target, output):
#     weights = np.array([0.4999014531185609, 1.023027280727486, 0.43079488604780436, 0.2869071613672823, 1.0034527406128615, 0.9122715233953503, 1.0209309133489461, 1.077802673259677, 1.222986893437952, 1.181302396477263, 1.1077582784086397, 0.8634563010646199, 0.8767519326252279, 0.6890930646117368])
#     target = tf.convert_to_tensor(target)
#     output = tf.convert_to_tensor(output)
#     target.shape.assert_is_compatible_with(output.shape)
#     weights = tf.reshape(tf.convert_to_tensor(weights, dtype=target.dtype), (1,-1))

#     # Adjust the predictions so that the probability of
#     # each class for every sample adds up to 1
#     # This is needed to ensure that the cross entropy is
#     # computed correctly.
#     output = output / tf.reduce_sum(output, -1, True)

#     # Compute cross entropy from probabilities.
#     epsilon_ = tf.constant(tf.keras.backend.epsilon(), output.dtype.base_dtype)
#     output = tf.clip_by_value(output, epsilon_, 1.0 - epsilon_)
#     return -tf.reduce_sum(weights * target * tf.math.log(output), axis=-1)

# del data 
# ros = RandomOverSampler(random_state=0)
# train_data = train_data.reshape((78324,784))
# train_data, train_labels = ros.fit_resample(train_data, train_labels)
# print(np.shape(train_data))


ValueError: Imbalanced-learn currently supports binary, multiclass and binarized encoded multiclasss targets. Multilabel and multioutput targets are not supported.

In [ ]:
cnn_models = [0]*class_num


In [ ]:
# train_data = train_data.astype('float32')
cnn_models[5].fit(split_train_images[5], split_labels[5], batch_size=64, epochs=40)
# for i in range(class_num):
#     print("Feature:", i)
#     cnn_models[i].fit(split_train_images[i], split_labels[i], batch_size=64, epochs=5)

Epoch 1/40
2312/2312 [==============================] - 21s 8ms/step - loss: 0.7055 - auc: 0.5397 - accuracy: 0.5306
Epoch 2/40
2312/2312 [==============================] - 19s 8ms/step - loss: 0.6839 - auc: 0.5767 - accuracy: 0.5559
Epoch 3/40
2312/2312 [==============================] - 19s 8ms/step - loss: 0.6790 - auc: 0.5935 - accuracy: 0.5703
Epoch 4/40
2312/2312 [==============================] - 18s 8ms/step - loss: 0.6748 - auc: 0.6061 - accuracy: 0.5775
Epoch 5/40
2312/2312 [==============================] - 19s 8ms/step - loss: 0.6713 - auc: 0.6147 - accuracy: 0.5838
Epoch 6/40
2312/2312 [==============================] - 19s 8ms/step - loss: 0.6683 - auc: 0.6218 - accuracy: 0.5887
Epoch 7/40
2312/2312 [==============================] - 19s 8ms/step - loss: 0.6651 - auc: 0.6290 - accuracy: 0.5933
Epoch 8/40
2312/2312 [==============================] - 18s 8ms/step - loss: 0.6626 - auc: 0.6343 - accuracy: 0.5958
Epoch 9/40
2312/2312 [==============================] - 18s 8ms/

KeyboardInterrupt: 